In [ ]:
import numpy as np
from obspy import read_events
from obspy import read_inventory
from obspy.clients.fdsn import Client
client_wm = Client("IRIS")
from obspy.clients.iris import Client  
from obspy.taup import TauPyModel
from obspy import Stream, Trace
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from obspy import UTCDateTime
import matplotlib.pyplot as plt
from obspy.geodetics import locations2degrees,degrees2kilometers
from scipy import signal
plt.rc('font', size=10) 
%matplotlib widget
import datetime
from obspy.signal.trigger import recursive_sta_lta,plot_trigger,coincidence_trigger
import pandas as pd

In [ ]:
inv0 = client_wm.get_stations(
    channel="BH*", starttime=UTCDateTime(2020,9,1,0,0,0),endtime=UTCDateTime(2020,10,1,0,0,0),
    network="AK", sta="SCRK,J25K,K27K",
    level="response")
the_station=inv0[0][2]
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&maxradius=4&minmagnitude=0'
cat1 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=4&maxradius=8&minmagnitude=1'
cat2 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=8&maxradius=20&minmagnitude=2'
cat3 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=20&maxradius=100&minmagnitude=3'
cat4 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=100&maxradius=180&minmagnitude=4'
cat5 = read_events(evid_dum)
cat_all=cat1+cat2+cat3+cat4+cat5
channel = "BHZ"
start_time=UTCDateTime(2020,9,1,0,0,0)
end_time=UTCDateTime(2020,9,10,0,0,0)
#st_2day = client_wm.get_waveforms("AK", "SCRK,K27K,J25K", "*", "BHZ", start_time, end_time)
st_10day = client_wm.get_waveforms("AK", "SCRK", "*", "BHZ", start_time, end_time)
EQ_data=pd.read_csv("detections/SCRK_outputs/X_prediction_results.csv")
eq_arv_t=EQ_data['event_start_time']
eq_arv_s_t=EQ_data['s_arrival_time']
st_10day2=st_10day.copy()
st_10day2.merge(method=0,fill_value=0)
st_10day2.detrend()
st_10day2.filter('bandpass',freqmin=0.8, freqmax=25.0)
trace=st_10day2[0]
model = TauPyModel(model="prem")
df = trace.stats.sampling_rate
npts = trace.stats.npts
t = np.arange(npts, dtype=np.float32) / df
cft = recursive_sta_lta(trace.data, int(5 * df), int(10 * df))
eqt_array=[]
for eqtm in eq_arv_t:
    dumt=UTCDateTime(eqtm)
    #ax.vlines(dumt-trace.stats.starttime, ii,jj, color='y',linestyles='dashdot', lw=3)
    eqt_array.append(dumt-trace.stats.starttime)
    picker_time_a=[]
for event in cat_all:
    disD=locations2degrees(the_station.latitude,the_station.longitude,event.origins[0].latitude,event.origins[0].longitude)
    if event.origins[0].depth<0:
        event.origins[0].depth=0
    arrivals = model.get_travel_times(source_depth_in_km=event.origins[0].depth/1000,
                                  distance_in_degree=disD)    
    picker_time_a.append(event.origins[0].time+arrivals[0].time-trace.stats.starttime)

max_search_range_b=10
max_search_range_a=30
cft_thr=1.35

sel_cat_list=[]

lij=-1
for event in cat_all:
    lij=lij+1
    disD=locations2degrees(the_station.latitude,the_station.longitude,event.origins[0].latitude,event.origins[0].longitude)
    picker_time=picker_time_a[lij]
    t0=int(picker_time/trace.stats.delta)
    tarray=range(int(t0-max_search_range_b*trace.stats.sampling_rate),int(t0+max_search_range_a*trace.stats.sampling_rate))
    try:
        if np.max(cft[tarray])>cft_thr:
            sel_cat_list.append(event)
            # if flag==0:
            #     cat_all_sel=event
            #     flag=1
            # else:
            #     cat_all_sel=cat_all_sel+event
    except Exception:
        continue
    
for i,event in enumerate(sel_cat_list):
    if i==0:
        dum=cat_all[0:1]
        dum.clear()
        all_sel_event=dum+event
    else:
        all_sel_event=all_sel_event+event
lij=-1
eqt_max_time=25
for i,picker_time in enumerate(picker_time_a):
    if np.abs(np.array(eqt_array)-picker_time).min()<eqt_max_time:
        lij=lij+1
        if lij==0:
            dum=cat_all[0:1]
            dum.clear()
            eqt_sel_event=dum+cat_all[i]
        else:
            eqt_sel_event=eqt_sel_event+cat_all[i]
eqtpall_sel_event=all_sel_event+eqt_sel_event
event_in_list=[]
for event in eqtpall_sel_event:
    disD=locations2degrees(the_station.latitude,the_station.longitude,event.origins[0].latitude,event.origins[0].longitude)
    event_list=[event.origins[0].longitude,event.origins[0].latitude,event.origins[0].depth,event.magnitudes[0].mag,disD,event.event_type,event.origins[0].time]
    event_in_list.append(event_list)
eqtpall_sel_event_pd=pd.DataFrame(event_in_list,columns=['longitude','latitude','depth','magnitude','disD','event_type','time'])
bh_catalog=eqtpall_sel_event_pd.drop_duplicates(subset=['longitude','latitude','magnitude'])

def threshold_trigger(th,record_time,trace):
    record_number=record_time*trace.stats.sampling_rate
    abs_st_data=np.abs(trace.data)
    ind_th=[]
    recording=False
    for i,abs_st in enumerate(abs_st_data):
        if abs_st>th and recording is False:
            ind_th.append(i)
            recording=True
        if ind_th!=[] and (i-ind_th[-1])>record_number:
            recording=False
    return len(ind_th),ind_th

def cal_bhcat_t(bh_catalog):
    cat_arl_t=[]
    cat_arl_i=[]
    for i in bh_catalog.index:
        arrivals = model.get_travel_times(source_depth_in_km=bh_catalog.loc[i,'depth']/1000,
                                    distance_in_degree=bh_catalog.loc[i,'disD'])    
        picker_time=bh_catalog.loc[i,'time']+arrivals[0].time-trace.stats.starttime
        cat_arl_t.append(picker_time)
        cat_arl_i.append(i)
    return cat_arl_t,cat_arl_i


def event_associate(pre_time,after_time,cat_arl_t,t,ind_th,cat_arl_i):
    n_recorded=0
    index_triggered=[]
    index_cat=[]
    for i in ind_th:
        for j,cat_t in enumerate(cat_arl_t):
            if cat_t>t[i]-pre_time and cat_t<t[i]+after_time:
                n_recorded=n_recorded+1
                index_triggered.append(cat_arl_i[j])
                index_cat.append(j)
                break
    return n_recorded,index_triggered,index_cat
def threshold_trigger_w_avg(th_fold,record_time,trace,window_len):
    record_number=record_time*trace.stats.sampling_rate
    abs_st_data=np.abs(trace.data)
    ind_th=[]
    recording=False
    rtt=0
    cm_sum=0
    ltavg=np.sum(abs_st_data[0:window_len])/window_len
    for i,abs_st in enumerate(abs_st_data):
        rtt=rtt+1
        if rtt>window_len:
            rtt=0
            ltavg=cm_sum/window_len
            #print(ltavg)
            cm_sum=abs_st
        cm_sum=cm_sum+abs_st
        if abs_st>th_fold*ltavg and recording is False:
            ind_th.append(i)
            recording=True
        if ind_th!=[] and (i-ind_th[-1])>record_number:
            recording=False
    return len(ind_th),ind_th
def cal_tauc(tr,sat_t,end_t):
    df = tr.stats.sampling_rate
    npts = tr.stats.npts
    t = np.arange(npts, dtype=np.float32) / df
    ind_range=(t<end_t) * (t>sat_t)
    r=np.sum(np.diff(tr[ind_range])**2/tr.stats.delta)/np.sum(tr[ind_range]**2)
    return 2*np.pi/np.sqrt(r)

In [ ]:
cat_arl_t,cat_arl_i=cal_bhcat_t(bh_catalog)

In [ ]:
pre_time=80
after_time=10
th_fold=200
th=200
record_time=60*30
fminay=np.arange(0.02,10,0.4)
window_len=50*60*2
freq_r_mat=np.zeros([25,75])
freq_r_mat_ww=np.zeros([25,75])
freq_r_r_mat_ww=np.zeros([25,75])
freq_r_r_mat=np.zeros([25,75])
for i,fmin in enumerate(fminay):
    fmaxay=np.arange(fmin+0.4,30,0.4)
    for j,fmax in enumerate(fmaxay):
        print(fmin,fmax)
        st_10day2=st_10day.copy()
        st_10day2.merge(method=0,fill_value=0)
        st_10day2.detrend()
        st_10day2.filter('bandpass',freqmin=fmin, freqmax=fmax)
        trace=st_10day2[0]

        n_dum,ind_th=threshold_trigger(th,record_time,trace)
        freq_r_mat[i][i+j]=n_dum
        n_recorded,index_triggered,index_cat1=event_associate(pre_time,after_time,cat_arl_t,t,ind_th,cat_arl_i)
        try:
            freq_r_r_mat[i][i+j]=n_recorded/n_dum
        except:
            freq_r_r_mat[i][i+j]=0
        n_dum,ind_th=threshold_trigger_w_avg(th_fold,record_time,trace,window_len)
        freq_r_mat_ww[i][i+j]=n_dum
        n_recorded,index_triggered,index_cat2=event_associate(pre_time,after_time,cat_arl_t,t,ind_th,cat_arl_i)
        try:
            freq_r_r_mat_ww[i][i+j]=n_recorded/n_dum
        except:
            freq_r_r_mat_ww[i][i+j]=0

In [ ]:
plt.rc('font', size=15) 

In [ ]:
fig=plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111)
#c = ax.contourf(range(60*5,60*60*2,60*5),range(100,1000,50),tri_matrix, 20,cmap='jet',norm=LogNorm(vmin=np.array(tri_matrix).min(), vmax=np.array(tri_matrix).max()))
c= ax.pcolor(np.arange(0.02,30,0.4),np.arange(0.42,10,0.4),freq_r_mat[:][1:],cmap='jet')

#ax.clabel(c, fmt='%2.1f', colors='w', fontsize=14)
ax.set_title('Threshold method number of triggers')
c.set_clim(0, 20)
cbar=fig.colorbar(c, ax=ax)
ax.set_xlabel('max frequency (Hz)')
ax.set_ylabel('min frequency (Hz)')
cbar.ax.set_ylabel('number of triggers')
ax.set_xlim(0,25)

In [ ]:
fig=plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111)
#c = ax.contourf(range(60*5,60*60*2,60*5),range(100,1000,50),tri_matrix, 20,cmap='jet',norm=LogNorm(vmin=np.array(tri_matrix).min(), vmax=np.array(tri_matrix).max()))
c= ax.pcolor(np.arange(0.02,30,0.4),np.arange(0.42,10,0.4),freq_r_mat_ww[:][1:],cmap='jet')

#ax.clabel(c, fmt='%2.1f', colors='w', fontsize=14)
ax.set_title('Segmented window method number of triggers')
c.set_clim(0, 20)
cbar=fig.colorbar(c, ax=ax)
ax.set_xlabel('max frequency (Hz)')
ax.set_ylabel('min frequency (Hz)')
cbar.ax.set_ylabel('number of triggers')
ax.set_xlim(0,25)

In [ ]:
fig=plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111)
#c = ax.contourf(range(60*5,60*60*2,60*5),range(100,1000,50),tri_matrix, 20,cmap='jet',norm=LogNorm(vmin=np.array(tri_matrix).min(), vmax=np.array(tri_matrix).max()))
c= ax.pcolor(np.arange(0.02,30,0.4),np.arange(0.42,10,0.4),freq_r_r_mat[:][1:],cmap='jet')

#ax.clabel(c, fmt='%2.1f', colors='w', fontsize=14)
ax.set_title('Threshold method correct ratio')
cbar=fig.colorbar(c, ax=ax)
c.set_clim(0, 1)
ax.set_xlabel('max frequency (Hz)')
ax.set_ylabel('min frequency (Hz)')
cbar.ax.set_ylabel('correct detection ratio')
ax.set_xlim(0,25)

In [ ]:
fig=plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111)
#c = ax.contourf(range(60*5,60*60*2,60*5),range(100,1000,50),tri_matrix, 20,cmap='jet',norm=LogNorm(vmin=np.array(tri_matrix).min(), vmax=np.array(tri_matrix).max()))
c= ax.pcolor(np.arange(0.02,30,0.4),np.arange(0.42,10,0.4),freq_r_r_mat_ww[:][1:],cmap='jet')

#ax.clabel(c, fmt='%2.1f', colors='w', fontsize=14)
ax.set_title('Segmented window method correct ratio')
cbar=fig.colorbar(c, ax=ax)
c.set_clim(0, 1)
ax.set_xlabel('max frequency (Hz)')
ax.set_ylabel('min frequency (Hz)')
cbar.ax.set_ylabel('correct detection ratio')
ax.set_xlim(0,25)

In [ ]:
from scipy.io import savemat

In [ ]:
savemat('freq_range_data.mat',{'freq_r_r_mat_ww':freq_r_r_mat_ww,'freq_r_r_mat':freq_r_r_mat,'freq_r_mat_ww':freq_r_mat_ww,'freq_r_mat':freq_r_mat})

In [ ]:
bh_catalog